# Notes

Ultimate goal: Predict phenotype

Q1: How to predict phenotype?

A1: linear model + covariates (testset) + PRS (testset)

Q2: Where is PRS come from?

A2: 
 
    genotype + ref_panel (infos.pos) -> LD matrix + correlation matrix
   
    LD matrix + sumstats(raw beta) -> heritability (consistency, flip)
    
    correlation matrix + heritability + sumstats -> adjusted betas
   
    adjusted betas * genotype (trainset) -> PRS (trainset)
    

Q3: Where is linear model comes from?

A3: Modeling on the `trainset`
   
    pheno (trainset) ~ covariates (trainset) + PRS (trainset)

# Command Interface

In [4]:
sos run ldpred.ipynb -h

usage: sos run ldpred.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  inf
  auto
  grid

Global Workflow Options:
  --datafolder 'pathname/for/data'
                        Data preparation folder that data is saved in # post-qc
  --hapmap3 'https://github.com/privefl/bigsnpr/raw/master/data-raw/hm3_variants.rds'
                        hapmap3 is used to filter out the SNPs in hapmap3
  --summstats-file 'path/to/sumstats/file'
                        path to summary statistics files, genotypes, phenotypes
                        and covaraites data "post-qc/Height.QC.gz"
  --bed-file 'path/to/bed/file'
                        "post-qc/EUR.QC.bed"
  --geno-file 'path/to/geno/fil

: 1

# Global Parameter Setting

In [ ]:
[global]

### Data preparation
# folder that data is saved in
# # post-qc
parameter: datafolder = "pathname/for/data" 
# hapmap3 is used to filter out the SNPs in hapmap3
parameter: hapmap3 = "https://github.com/privefl/bigsnpr/raw/master/data-raw/hm3_variants.rds"
# path to summary statistics files, genotypes, phenotypes and covaraites data
# "post-qc/Height.QC.gz"
parameter: summstats_file =  "path/to/sumstats/file" 
# "post-qc/EUR.QC.bed"
parameter: bed_file =  "path/to/bed/file" 
# "post-qc/EUR.QC.rds"
parameter: geno_file = "path/to/geno/file" 
# "post-qc/EUR.height"
parameter: pheno_file = "path/to/pheno/data"
# "post-qc/EUR.cov","post-qc/EUR.eigenvec"
parameter: covariate_file = "path/to/cov/data"
# this need to be generalized, fix this
parameter: pcs_file = "path/to/pc/data"
parameter: tmpdir = "tmp-dataset"

# new data for predict PRS
parameter: newgeno = "path/to/new/genodata" ##  contains genotype data 
parameter: newcov = "path/to/new/covariates/data" ##  contains covariates (sex and PC in this case)

# parameters that dave
parameter: pheno_out = 'path/PhenoOut.RData'
parameter: summary_stat = 'path/SumStats.RData'
parameter: ld_in = 'path/LdInput.RData'
parameter: ld_out = 'path/LdOutput.RData'
parameter: ldreg_out = 'path/LdRegOut.RData'
parameter: inf_beta = 'path/InfBeta.RData'
parameter: grid_beta = 'path/GridBeta.RData'
parameter: auto_beta = 'path/AutoBeta.RData'
parameter: inf_prs = 'path/InfPrs.RData'
parameter: grid_prs = 'path/GridPrs.RData'
parameter: auto_prs = 'path/AutoPrs.RData'
parameter: null_r2 = 'path/NullR2.RData'
parameter: inf_mod = 'path/InfMod.RData'
parameter: grid_mod = 'path/GridMod.RData'
parameter: auto_mod = 'path/AutoMod.RData'
parameter: inf_pheno_pred = 'path/InfPhenoPred.RData'
parameter: grid_pheno_pred = 'path/GridPhenoPred.RData'
parameter: auto_pheno_pred = 'path/AutoPhenoPred.RData'




# Initialize variables for storing the LD score and LD matrix
corr = NULL
ld = NULL
# We want to know the ordering of samples in the bed file 
fam.order = NULL


# Workflow

## Prepare workflows

In [ ]:
[inf_10, auto_10, grid_10]

R: expand=True
    ## prepare workplace ##
    library(bigsnpr)
    # options(bigstatsr.check.parallel.blas = FALSE)
    # options(default.nproc.blas = NULL)
    library(data.table)
    library(magrittr)

## Load reference panel and summary statistics

save `sumstats` as `SumStats.RData`

In [ ]:
[inf_20, auto_20, grid_20]

input: hapmap = hapmap3, sum = summstats_file
output: summary_stat

R: expand=True
    ## Prep data ##
    info <- readRDS(url("{_input["hapmap"]}"))
    # Read in the summary statistic file
    sumstats <- bigreadr::fread2("{_input["sum"]}") 
    # LDpred 2 require the header to follow the exact naming
    names(sumstats) <-
        c("chr",
        "pos",
        "rsid",
        "a1",
        "a0",
        "n_eff",
        "beta_se",
        "p",
        "OR",
        "INFO",
        "MAF")
    # Transform the OR into log(OR)
    sumstats$beta <- log(sumstats$OR)
    # Filter out hapmap SNPs
    sumstats <- sumstats[sumstats$rsid %in% info$rsid,]
    save(sumstats, file = "{_output}")

## Read in phenotype and covariate files

save `pheno` as `PhenoOut.RData`

In [ ]:
[inf_25, auto_25, grid_25]


output: pheno_out

R: expand=True

    phenotype <- fread("{pheno_file}")
    covariate <- fread("{covariate_file}")
    pcs <- fread("{pcs_file}")
    # rename columns
    colnames(pcs) <- c("FID","IID", paste0("PC",1:6))
    # generate required table
    pheno <- merge(phenotype, covariate) %>%
        merge(., pcs)
    save(pheno, file = "{_output}")

## Preprocess the bed file

In [ ]:
[inf_30, auto_30, grid_30]

input: bed_file

# what is the output for snp_readBed 

R: expand=True 
    if(length(list.files(path = datafolder, pattern = "\\.bk$")) = 0){
    # preprocess the bed file (only need to do once for each data set)
    snp_readBed("{_input}")
    }


## SNP matching and get the CM information from 1000 Genome

* Perform SNP matching `snp_match(sumstats, map)` to get `info_snp`

Mathch alleles between summary statistics `sumstats` and SNP information from `obj.bigSNP`.

* CM information from 1000 Genome `snp_asGeneticPos(CHR, POS, dir = ".")`

Use genetic maps available at https://github.com/joepickrell/1000-genomes-genetic-maps/ to interpolate physical positions (in bp) to genetic positions (in cM).


save `obj.bigSNP`, `genotype`, `map`, `CHR`, `POS`, `info_snp` and `POS2` as `LdInput.RData`

In [ ]:
[inf_31, auto_31, grid_31]

input: geno = geno_file, sums = summary_stat
output: ld_in
 
R: expand=True 

    load("{_input["sums"]}")
    # now attach the genotype object
    obj.bigSNP <- snp_attach("{_input["geno"]}")
    
    # Assign the genotype to a variable for easier downstream analysis
    genotype <- obj.bigSNP$genotypes
    
    # extract the SNP information from the genotype
    map <- obj.bigSNP$map[-3]
    names(map) <- c("chr", "rsid", "pos", "a1", "a0")  
    
    # Rename the data structures
    CHR <- map$chr
    POS <- map$pos   

    # perform SNP matching
    info_snp <- snp_match(sumstats, map)
    
    # get the CM information from 1000 Genome
    # will download the 1000G file to the current directory (".")
    POS2 <- snp_asGeneticPos(CHR, POS, dir = ".")
    
    # save data to Rdata file
    save(obj.bigSNP, genotype, map, CHR, POS, info_snp, POS2, file = "{_output}")

## Calculate LD

* calculate LD using genotype from `obj.bigSNP` and CM information (distance)

save `info_snp`, `ld`, `fam.order`, `corr`, `NCORES`, `genotype` as `LdOutput.RData`

In [ ]:
[inf_32, auto_32, grid_32]

input: ld_in
output: ld_out

R: expand = True
    load("{_input}")
    # calculate LD
    # Get maximum amount of cores
    NCORES <- nb_cores()  
    # Open a temporary file
    tmp <- tempfile(tmpdir = tmpdir)
    on.exit(file.remove(paste0(tmp, ".sbk")), add = TRUE)
    for (chr in 1:22) {
        # Extract SNPs that are included in the chromosome
        ind.chr <- which(info_snp$chr == chr)
        ind.chr2 <- info_snp$`_NUM_ID_`[ind.chr]
        # Calculate the LD
        corr0 <- snp_cor(
                genotype,
                ind.col = ind.chr2,
                ncores = NCORES,
                infos.pos = POS2[ind.chr2],
                size = 3 / 1000
            )
        if (chr == 1) {
            ld <- Matrix::colSums(corr0^2)
            corr <- as_SFBM(corr0, tmp)
        } else {
            ld <- c(ld, Matrix::colSums(corr0^2))
            corr$add_columns(corr0, nrow(corr))
        }
    }
    # We assume the fam order is the same across different chromosomes
    fam.order <- as.data.table(obj.bigSNP$fam)
    # Rename fam order
    setnames(fam.order,
            c("family.ID", "sample.ID"),
            c("FID", "IID"))
    # save results
    save(info_snp, ld, fam.order, corr, NCORES, genotype, file = "{_output}")

## Perform LD score regression 

Using funciton `snp_ldsc()` to obtain $h^2$ the (SNP) heritability

save `h2_est`, `df_beta`, `corr`, `NCORES`, `info_snp`, `genotype` as `LdRegOut.RData` 

In [ ]:
[inf_40, auto_40, grid_40]

input: ld_out
output: ldreg_out

R: expand=True
    load("{_input}")
    df_beta <- info_snp[,c("beta", "beta_se", "n_eff", "_NUM_ID_")]
    ldsc <- snp_ldsc(   ld, 
                    length(ld), 
                    chi2 = (df_beta$beta / df_beta$beta_se)^2,
                    sample_size = df_beta$n_eff, 
                    blocks = NULL)
    h2_est <- ldsc[["h2"]]
    save(h2_est, df_beta, corr, NCORES, info_snp, genotype,file = "{_output}")

## Get adjusted betas

### Infinitesimal model

save `beta_inf`, `df_beta`, `corr`, `NCORES`,`info_snp`, `genotype` as `InfBeta.RData`

In [ ]:
[inf_50]

input: ldreg_out
output: inf_beta

R: expand=True
    load("{_input}")
    ## adjusted beta ##
    beta_inf <- snp_ldpred2_inf(corr, df_beta, h2 = h2_est)
    # save data
    save(beta_inf, df_beta, corr, NCORES,info_snp, genotype, file = "{_output}")

### Grid model

save `beta_grid`, `df_beta`, `corr`, `NCORES`,`info_snp`, `genotype` as `GridBeta.Rdata`

In [ ]:
[grid_50]

input: ldreg_out
output: grid_beta

R: expand=True
    load("{_input}")
    # Prepare data for grid model
    p_seq <- signif(seq_log(1e-4, 1, length.out = 17), 2)
    h2_seq <- round(h2_est * c(0.7, 1, 1.4), 4)
    grid.param <-
        expand.grid(p = p_seq,
                h2 = h2_seq,
                sparse = c(FALSE, TRUE))
    # Get adjusted beta from grid model
    beta_grid <- snp_ldpred2_grid(corr, df_beta, grid.param, ncores = NCORES)
    # save data
    save(beta_grid, df_beta, corr, NCORES,info_snp, genotype, file = "{_output}")

### Auto model

save `beta_auto`, `df_beta`, `corr`, `NCORES`,`info_snp`, `genotype` as `AutoBeta.Rdata`

In [ ]:
[auto_50]

input: ldreg_out
output: auto_beta

R: expand=True

    load("{_input}")
    # Get adjusted beta from the auto model
    multi_auto <- snp_ldpred2_auto(
        corr,
        df_beta,
        h2_init = h2_est,
        vec_p_init = seq_log(1e-4, 0.9, length.out = NCORES),
        ncores = NCORES
    )
    beta_auto <- sapply(multi_auto, function(auto)
        auto$beta_est)
    # save data
    save(beta_auto, df_beta, corr, NCORES,info_snp, genotype, file = "{_output}")

## Get PRS

### Infinitesimal model

save `pred_inf` as `InfPrs.RData`

In [ ]:
[inf_60]

input: inf_beta
output: inf_prs

R: expand=True
    
    load("{_input}")
    # calculate PRS for all samples
    ind.test <- 1:nrow(genotype)
    pred_inf <- big_prodVec(    genotype,
                                beta_inf,
                                ind.row = ind.test,
                                ind.col = info_snp$`_NUM_ID_`)
    save(pred_inf, file = "{_output}")

### Grid model
save `pred_grid` as `GridPrs.RData`

In [ ]:
[grid_60]

input: grid_beta
output: grid_prs

R: expand=True

    load("{_input}")
    ind.test <- 1:nrow(genotype)
    pred_grid <- big_prodMat(   genotype, 
                                beta_grid, 
                                ind.col = info_snp$`_NUM_ID_`)
    save(pred_grid, file = "{_output}")

### Auto model

save `pred_auto` as `AutoPrs.RData`

In [ ]:
[auto_60]

input: auto_beta
output: auto_prs

R: expand=True
    
    load("{_input}")
    # calculate PRS for all samples
    ind.test <- 1:nrow(genotype)
    pred_auto <-
        big_prodMat(genotype,
                    beta_auto,
                    ind.row = ind.test,
                    ind.col = info_snp$`_NUM_ID_`)
    # scale the PRS generated from AUTO
    pred_scaled <- apply(pred_auto, 2, sd)
    final_beta_auto <-
        rowMeans(beta_auto[,
                    abs(pred_scaled -
                        median(pred_scaled)) <
                        3 * mad(pred_scaled)])
    pred_auto <-
        big_prodVec(genotype,
            final_beta_auto,
            ind.row = ind.test,
            ind.col = info_snp$`_NUM_ID_`)
    save(pred_auto, file = "{_output}")

## Performance of LDpred model

### Calculate null $R^2$

save `null.r2` ,`y` as `NullR2.RData`

In [ ]:
[inf_70, auto_70, grid_70]
 
input: pheno_out
output: null_r2

R: expand=True

    load("{_input}")
    # Reformat the phenotype file such that y is of the same order as the 
    # sample ordering in the genotype file
    y <- pheno[fam.order, on = c("FID", "IID")]
    # Calculate the null R2
    # use glm for binary trait 
    # (will also need the fmsb package to calculate the pseudo R2)
    null.model <- paste("PC", 1:6, sep = "", collapse = "+") %>%
        paste0("Height~Sex+", .) %>%
        as.formula %>%
        lm(., data = y) %>%
        summary
    null.r2 <- null.model$r.squared
    save(null.r2,y, file = "{_output}")

### calculate model $R^2$

#### Infinitesimal model

save `inf.model`, `result` (R2) as `InfMod.Rdata`

In [ ]:
[inf_80]

input: r2 = null_r2, prs = inf_prs
output: inf_mod

R: expand=True
    load("{_input["r2"]}")
    load("{_input["prs"]}")
    reg.formula <- paste("PC", 1:6, sep = "", collapse = "+") %>%
        paste0("Height~PRS+Sex+", .) %>%
        as.formula
    reg.dat <- y
    reg.dat$PRS <- pred_inf
    inf.model <- lm(reg.formula, dat=reg.dat)
    result <- data.table(
        infinitesimal = inf.model$r.squared - null.r2,
        null = null.r2)
    save(inf.model,result, file = "{_output}")

#### Grid model

save `grid.model`, `result` (R2) as `GridMod.Rdata`

In [ ]:
[grid_80]

input: r2 = null_r2, prs = inf_prs
output: grid_mod

R: expand=True

    load("{_input["r2"]}")
    load("{_input["prs"]}")
    reg.formula <- paste("PC", 1:6, sep = "", collapse = "+") %>%
        paste0("Height~PRS+Sex+", .) %>%
        as.formula
    reg.dat <- y
    max.r2 <- 0
    for(i in 1:ncol(pred_grid)){
        reg.dat$PRS <- pred_grid[,i]
        grid.model <- lm(reg.formula, dat=reg.dat)
        if(max.r2 < grid.model$r.squared){
            max.r2 <- grid.model$r.squared
        }
    }
    result <- data.table(
    grid = max.r2 - null.r2,
    null = null.r2)
    save(grid.model,result, file = "{_output}")

#### Auto model

save `auto.model`, `result` (R2) as `AutoMod.Rdata`

In [ ]:
[auto_80]

input: r2 = null_r2, prs = inf_prs
output: auto_mod

R: expand=True

    load("{_input["r2"]}")
    load("{_input["prs"]}")
    reg.formula <- paste("PC", 1:6, sep = "", collapse = "+") %>%
        paste0("Height~PRS+Sex+", .) %>%
        as.formula
    reg.dat <- y
    reg.dat$PRS <- pred_auto
    auto.model <- lm(reg.formula, dat=reg.dat)
    result <- data.table(
            auto = auto.model$r.squared - null.r2,
            null = null.r2)
    save(auto.model,result, file = "{_output}")

## Predict phenotype

fix this. xxx stands for different model

In [ ]:
[inf_90]

input: model = inf_mod, ngeno = newgeno,nconv = newcov, adjbeta = inf_beta
output: inf_pheno_pred

R: expend = True
    load("{_input["model"]}")
    new_geno = fread("{_input["ngeno"]}")
    new_cov = fread("{_input["ncov"]}")
    # predict prs
    ind.test <- 1:nrow(new_geno)
    prs_pred <- big_prodVec(new_geno,
                            beta_inf,
                            ind.row = ind.test,
                            ind.col = info_snp$`_NUM_ID_`)
    new_cov$PRS = prs_pred
    pheno_pred = predict(inf.model,newdata = new_cov)
    save(prs_pred,pheno_pred, file = "{_output}")

In [ ]:
[grid_90]

input: model = grid_mod, ngeno = newgeno,nconv = newcov, adjbeta = grid_beta
output: grid_pheno_pred

R: expend = True
    load("{_input["model"]}")
    new_geno = fread("{_input["ngeno"]}")
    new_cov = fread("{_input["ncov"]}")
    # predict prs
    ind.test <- 1:nrow(new_geno)
    prs_pred <- big_prodVec(new_geno,
                            beta_grid,
                            ind.row = ind.test,
                            ind.col = info_snp$`_NUM_ID_`)
    new_cov$PRS = prs_pred
    pheno_pred = predict(inf.model,newdata = new_cov)
    save(prs_pred,pheno_pred, file = "{_output}")

In [ ]:
[auto_90]

input: model = auto_mod, ngeno = newgeno,nconv = newcov, adjbeta = auto_beta
output: auto_pheno_pred

R: expend = True
    load("{_input["model"]}")
    new_geno = fread("{_input["ngeno"]}")
    new_cov = fread("{_input["ncov"]}")
    # predict prs
    ind.test <- 1:nrow(new_geno)
    prs_pred <- big_prodVec(new_geno,
                            beta_auto,
                            ind.row = ind.test,
                            ind.col = info_snp$`_NUM_ID_`)
    new_cov$PRS = prs_pred
    pheno_pred = predict(inf.model,newdata = new_cov)
    save(prs_pred, pheno_pred, file = "{_output}")

## Model fitting and prediction results

Return fitted model, adjusted beta, predicted PRS and predicted phenotype

In [ ]:
[inf_100]

input: mod = inf_mod, beta = inf_beta, pred = inf_pheno_pred
output: inf_ldpred_res

R: expend = True
    load("{_input["mod"]}")
    load("{_input["beta"]}")
    load("{_input["pred"]}")
    ldpred_res = list(model = inf.model, 
                      R2 = result,
                      adj_beta = beta_inf,
                      pred_prs = prs_pred,
                      pred_pheno = pheno_pred)
    save(ldpred_res, file = "{_output}")

In [ ]:
[grid_100]

input: mod = grid_mod, beta = grid_beta, pred = grid_pheno_pred
output: grid_ldpred_res

R: expend = True
    load("{_input["mod"]}")
    load("{_input["beta"]}")
    load("{_input["pred"]}")
    ldpred_res = list(model = grid.model, 
                      R2 = result,
                      adj_beta = beta_grid,
                      pred_prs = prs_pred,
                      pred_pheno = pheno_pred)
    save(ldpred_res, file = "{_output}")

In [ ]:
[auto_100]

input: mod = auto_mod, beta = auto_beta, pred = auto_pheno_pred
output: auto_ldpred_res

R: expend = True
    load("{_input["mod"]}")
    load("{_input["beta"]}")
    load("{_input["pred"]}")
    ldpred_res = list(model = auto.model, 
                      R2 = result,
                      adj_beta = beta_auto,
                      pred_prs = prs_pred,
                      pred_pheno = pheno_pred)
    save(ldpred_res, file = "{_output}")